In [1]:
from enum import Enum
from collections import namedtuple

ChainType = namedtuple('ChainType', ['id', 'vault_address'])

class Chain(Enum):

    @property
    def vault_address(self):
        return self.value.vault_address
    
    @property
    def id(self):
        return self.value.id
    
    mainnet = ChainType(1, "0xBA12222222228d8Ba445958a75a0704d566BF2C8")
    polygon = ChainType(137, "0xBA12222222228d8Ba445958a75a0704d566BF2C8")
    arbitrum = ChainType(42161, "0xBA12222222228d8Ba445958a75a0704d566BF2C8")
    gnosis = ChainType(100, "0xBA12222222228d8Ba445958a75a0704d566BF2C8")
    optimism = ChainType(10, "0xBA12222222228d8Ba445958a75a0704d566BF2C8")
    goerli = ChainType(5, "0xBA12222222228d8Ba445958a75a0704d566BF2C8")

In [2]:
import json
import os
from functools import cache

import requests as r
from web3 import Web3


@cache
def get_web3_instance():
    return Web3(
        Web3.HTTPProvider(
            f"https://eth.llamarpc.com/rpc/{os.getenv('LLAMA_PROJECT_ID')}"
        )
    )


@cache
def get_abi_from_etherscan(contract_address):
    response = r.get(
        f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}"
    ).json()
    return json.loads(response["result"])


@cache
def get_web3_contract(contract_address):
    w3 = get_web3_instance()
    return w3.eth.contract(
        address=w3.to_checksum_address(contract_address),
        abi=get_abi_from_etherscan(contract_address),
    )

In [4]:
# Easy functions

class Vault:
    def __init__(self, chain=Chain.mainnet, private_key=None):
        self.chain = chain
        self.contract = get_web3_contract(chain.vault_address)

    def get_authorizer(self):
        return self.contract.functions.getAuthorizer().call()
    
    def get_protocol_fees_collector(self):
        return self.contract.functions.getProtocolFeesCollector().call()

    def has_approved_relayer(self, user_address:str, relayer_address:str):
        return self.contract.functions.hasApprovedRelayer(user_address, relayer_address).call()  

    def set_relayer_approval(
            self, 
            sender_address:str, 
            relayer_address:str, 
            approval:bool, 
            gasFactor=1.05, 
            gasPriceSpeed="average", 
            nonceOverride=-1, 
            gasEstimateOverride=-1, 
            gasPriceGweiOverride=-1
        ):

        set_relayer_approval = self.contract.functions.setRelayerApproval(sender_address, relayer_address, approval)
        tx = send
        return self.contract.functions.setRelayerApproval(sender_address, relayer_address, approval).call()

    def get_pool(self, pool_id:str):
        return self.contract.functions.getPool(pool_id).call()
    
    def get_pool_token_info(self, pool_id, token_address):
        return self.contract.functions.getPoolTokenInfo(pool_id, token_address).call()
    
    def set_paused(self, paused):
        return self.contract.functions.setPaused(paused).call()

In [5]:
## Read functions
vault = Vault()
print(vault.get_authorizer())
print(vault.get_protocol_fees_collector())
print(vault.has_approved_relayer("0x62dc515c4f2239597Cedfe7F8260f72d8f14A960", "0xC92E8bdf79f0507f65a392b0ab4667716BFE0110"))
print(vault.set_relayer_approval("0x62dc515c4f2239597Cedfe7F8260f72d8f14A960", "0xC92E8bdf79f0507f65a392b0ab4667716BFE0110", True))
# print(vault.get_pool())
# print(vault.get_pool_token_info())


# BalancerRelayer: 
# VaultRelayer: 
# Which one I have to use?

0xA331D84eC860Bf466b4CdCcFb4aC09a1B43F3aE6
0xce88686553686DA562CE7Cea497CE749DA109f9F
True


In [ ]:
# Exit flow on the FE
# https://github.com/balancer/balancer-sdk/blob/aa6b4dc3a02138f878feabd98ec72e8f06b4e68d/balancer-js/examples/exitGeneralised.ts#L185
# exit_pool parameters
# poolId (bytes32)
#   poolId (bytes32)
# sender (address)
#   sender (address)
# recipient (address)
#   recipient (address)
# request (tuple)
    # assets
    # assets (address[])
    # minAmountsOut (or limits in the PoolBalanceChange struct)
    # could probably be minAmountsOut: new Array(assets.length).fill('0'),
    # see https://github.com/balancer/balancer-sdk/blob/aa6b4dc3a02138f878feabd98ec72e8f06b4e68d/balancer-js/src/modules/relayer/actions.ts#L87
    # minAmountsOut (uint256[])
    # userData -- this is built by hand, encodes the following:
#       // Exit pool proportionally or to a singleToken
#   if (exitTokenIndex > -1) {
#     userData = StablePoolEncoder.exitExactBPTInForOneTokenOut(
#       amount,
#       exitTokenIndex
#     );
#   } else {
#     const encoder = isComposable
#       ? ComposableStablePoolEncoder.exitExactBPTInForAllTokensOut
#       : StablePoolEncoder.exitExactBPTInForTokensOut;
#     userData = encoder(amount);
#   }
    # userData (bytes)
    # toInternalBalance
    # toInternalBalance (bool)
    def exit_pool(self, poolId, recipient, assets, min_amounts_out, user_data, to_internal_balance):
        # Calls _joinOrExit(PoolBalanceChangeKind.EXIT, poolId, sender, recipient, _toPoolBalanceChange(request));
        # in PoolBalances contract
        # What is _toPoolBalanceChange(request)?
        # calls pool.onExitPool
        # // The Vault ignores the `recipient` in joins and the `sender` in exits: it is up to the Pool to keep track of
        # // their participation.
        # calls _processExitPoolTransfers
        # The vault ignores the sender so we can send anything here
        # request is turned into _toPoolBalanceChange(request)
        # request type first is ExitPoolRequest, and _toPoolBalanceChange returns PoolBalanceChange
        return self.contract.functions.exitPool(poolId, recipient, recipient, (assets, min_amounts_out, user_data, to_internal_balance)).call()